In [7]:
import pandas as pd
from textblob import TextBlob
import numpy as np
from datetime import datetime
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics as m
from sklearn.preprocessing import OneHotEncoder
import nltk, sys, json
import discogs_client
from urllib import request
from urllib.parse import parse_qsl
from urllib.parse import urlparse
import oauth2 as oauth

data = pd.read_csv('Record-Collection.csv')
consumer_key = 'FdxaljOTWBmKXCysYsQZ'
consumer_secret = 'lrfQIsgQonpeUnEuLWRpJgWKLdhKMaYe'
request_token_url = 'https://api.discogs.com/oauth/request_token'
authorize_url = 'https://www.discogs.com/oauth/authorize'
access_token_url = 'https://api.discogs.com/oauth/access_token'
user_agent = 'discogs_api_example/1.0'
consumer = oauth.Consumer(consumer_key, consumer_secret)
client = oauth.Client(consumer)
resp, content = client.request(request_token_url, 'POST', headers={'User-Agent': user_agent})
if resp['status'] != '200':
    sys.exit('Invalid response {0}.'.format(resp['status']))
request_token = dict(parse_qsl(content.decode('utf-8')))

print(' == Request Token == ')
print(f'    * oauth_token        = {request_token["oauth_token"]}')
print(f'    * oauth_token_secret = {request_token["oauth_token_secret"]}')
print()
print(f'Please browse to the following URL {authorize_url}?oauth_token={request_token["oauth_token"]}')

# Waiting for user input
accepted = 'n'
while accepted.lower() == 'n':
    print()
    accepted = input(f'Have you authorized me at {authorize_url}?oauth_token={request_token["oauth_token"]} [y/n] :')
oauth_verifier = input('Verification code : ')

# Generate objects that pass the verification key with the oauth token and oauth
# secret to the discogs access_token_url
token = oauth.Token(request_token['oauth_token'], request_token['oauth_token_secret'])
token.set_verifier(oauth_verifier)
client = oauth.Client(consumer, token)

resp, content = client.request(access_token_url, 'POST', headers={'User-Agent': user_agent})
access_token = dict(parse_qsl(content.decode('utf-8')))

print(' == Access Token ==')
print(f'    * oauth_token        = {access_token["oauth_token"]}')
print(f'    * oauth_token_secret = {access_token["oauth_token_secret"]}')
print(' Authentication complete. Future requests must be signed with the above tokens.')
print()

token = oauth.Token(key=access_token['oauth_token'],
        secret=access_token['oauth_token_secret'])
client = oauth.Client(consumer, token)

 == Request Token == 
    * oauth_token        = xflYnKShWsPiedZgOOgKgrZcOMQxqjEHKnIqwSnx
    * oauth_token_secret = sALaBeDhqYjMiDkDfDGOYEegUdRVyQeRBUZEeOUH

Please browse to the following URL https://www.discogs.com/oauth/authorize?oauth_token=xflYnKShWsPiedZgOOgKgrZcOMQxqjEHKnIqwSnx

 == Access Token ==
    * oauth_token        = mVwHQbCqiKdBUWMTwwAKRpxIztrgqntADsKjmnBe
    * oauth_token_secret = lnqIqazCbaajQNvXKAWNptEYvZJNbuQrJGKSPjLj
 Authentication complete. Future requests must be signed with the above tokens.


== Search results for release_title= Deft Ones, Artist= Commodo ==
 == API image request ==
    * response status      = 200
    * saving image to disk = anBn.jpeg


In [17]:
# With an active auth token, we're able to reuse the client object and request
# additional discogs authenticated endpoints, such as database search.

artist_ = 'Jade Cicada'
title_ = 'Brood VII'

resp, content = client.request('https://api.discogs.com/database/search?release_title='+title_.replace(" ",'+')+'&artist='+artist_.replace(" ",'+'),
        headers={'User-Agent': user_agent})

if resp['status'] != '200':
    sys.exit('Invalid API response {0}.'.format(resp['status']))

releases = json.loads(content.decode('utf-8'))
print('\n== Search results for release_title= ' + title_ + ', Artist= ' + artist_ + ' ==')
for release in releases['results']:
    print(f'\n\t== discogs-id {release["id"]} ==')
    print(f'\tTitle\t: {release.get("title", "Unknown")}')
    print(f'\tYear\t: {release.get("year", "Unknown")}')
    print(f'\tLabels\t: {", ".join(release.get("label", ["Unknown"]))}')
    print(f'\tCat No\t: {release.get("catno", "Unknown")}')
    print(f'\tFormats\t: {", ".join(release.get("format", ["Unknown"]))}')

resp, content = client.request('https://api.discogs.com/releases/40522',
        headers={'User-Agent': user_agent})

if resp['status'] != '200':
    sys.exit('Unable to fetch release 40522')

# load the JSON response content into a dictionary.
release = json.loads(content.decode('utf-8'))
# extract the first image uri.
image = release['images'][0]['uri']

# The authenticated URL is generated for you. There is no longer a need to
# wrap the image download request in an OAuth signature.
# build, send the HTTP GET request for the desired image.
# DOCS: http://www.discogs.com/forum/thread/410594
try:
    request.urlretrieve(image, image.split('/')[-1])
except Exception as e:
    sys.exit(f'Unable to download image {image}, error {e}')

print(' == API image request ==')
print(f'    * response status      = {resp["status"]}')
print(f'    * saving image to disk = {image.split("/")[-1]}')






== Search results for release_title= Brood VII, Artist= Jade Cicada ==

	== discogs-id 17109261 ==
	Title	: Jade Cicada - Brood VII
	Year	: 2017
	Labels	: Swarm Audio
	Cat No	: none
	Formats	: Vinyl, 12", 45 RPM, Album, Compilation

	== discogs-id 15522483 ==
	Title	: Jade Cicada - Brood VII
	Year	: 2017
	Labels	: Swarm Audio
	Cat No	: none
	Formats	: File, AAC, AIFF, FLAC, MP3, WAV, Album, Vinyl, LP, Limited Edition
 == API image request ==
    * response status      = 200
    * saving image to disk = anBn.jpeg


In [ ]:

case_num, description, sentiment_score, polarity_score, type, subtype, rating, dates = [], [], [], [], [], [], [], []
complaint_type_ , MSE_LR, MSE_XGB, R2_LR, R2_XGB = [], [], [], [], []
x = 0
while x < len(data):
    description_x = str(data.iat[x,data.columns.get_loc('Description of the Problem/Request')]).strip()
    type_x = str(data.iat[x,data.columns.get_loc('Concern Type')]).strip()
    subtype_x = str(data.iat[x,data.columns.get_loc('Concern Sub-Type')]).strip()
    case_num_x = str(data.iat[x,data.columns.get_loc('Case Number')]).strip()
    sentiment_score_x = TextBlob(description_x).sentiment.subjectivity    
    polarity_score_x = TextBlob(description_x).sentiment.polarity    
    sentiment_category = [1 if sentiment_score_x > 0                      
                          else -1 if sentiment_score_x < 0                        
                          else 0]
    case_num.append(case_num_x)
    description.append(description_x)
    sentiment_score.append(sentiment_score_x)
    polarity_score.append(polarity_score_x)
    rating.append(sentiment_category)
    type.append(type_x)
    subtype.append(subtype_x)
    dates.append(data.iat[x,data.columns.get_loc('Opened Date')])
    x = x + 1
# pre process data for training model
day_of_week = [datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S').weekday() for date in dates]
results = {  'Case Number': case_num  
            ,'Day of Week' : day_of_week  
            ,'Type': type          
            ,'Sub-Type': subtype          
            ,'Description': description          
            ,'Sentiment': sentiment_score          
            ,'Polarity': polarity_score          
            ,'Rating Score': rating
            }

pre_results_df = pd.DataFrame(results)
encoder = OneHotEncoder(handle_unknown='ignore')
x_encoded = encoder.fit_transform(pre_results_df['Type'].values.reshape(-1,1))
x_encoded_df = pd.DataFrame(x_encoded.toarray(), columns=encoder.get_feature_names_out(['Type']))
encoded_df = pd.concat([pre_results_df, x_encoded_df], axis=1)
encoded_df = encoded_df.drop(['Type'], axis=1)
x_encoded = encoder.fit_transform(encoded_df['Day of Week'].values.reshape(-1,1))
x_encoded_df = pd.DataFrame(x_encoded.toarray(), columns=encoder.get_feature_names_out(['Day of Week']))
encoded_df = pd.concat([encoded_df, x_encoded_df], axis=1)
encoded_df = encoded_df.drop(['Day of Week'], axis=1)
complaint_types = encoded_df.columns.tolist()

# create training and testing models for each complaint type
x = 6
while x < len(complaint_types):
    complaint_type_.append(complaint_types[x])
    # split complaint data into training / test sets    
    features = [complaint_types[x], 'Sentiment' ]
    target = "Polarity"
    x_train, x_test, y_train, y_test = train_test_split(encoded_df[features], encoded_df[target], test_size=0.3, random_state=0)

    # train the prediction models    
    regressor = LinearRegression()
    x_regressor = XGBRegressor(random_state=42)
    y_train = y_train.values.reshape(-1,1)
    regressor.fit(x_train, y_train)
    x_regressor.fit(x_train,y_train)

    # make prediction using test data    
    M_pred = regressor.predict(x_test)
    X_pred = x_regressor.predict(x_test)

    # evaluate model performance    
    mse = m.mean_squared_error(y_test, M_pred)
    R2_ = m.r2_score(y_test,M_pred)
    MSE_LR.append(mse)
    R2_LR.append(R2_)
    mse_ = m.mean_squared_error(y_test,X_pred)
    MSE_XGB.append(mse_)
    R2__ = m.r2_score(y_test,X_pred)
    R2_XGB.append(R2__)
    x = x + 1

trained_results = { 'Complaint Type' : complaint_type_
                  ,'MSE LR' : MSE_LR
                  ,'R2 LR': R2_LR
                  ,'MSE X' : MSE_XGB
                  ,'R2 X' : R2_XGB                  
                  }

trained_results_df = pd.DataFrame(trained_results)